In [ ]:
!pip freeze > requirements.txt

In [ ]:
!which python
!python --version

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
import tempfile


In [ ]:
# Create a ChatOpenAI model for Gemma
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    temperature=0,
    api_key="not-needed",
    model_name="llama3"
)

In [ ]:
# Create a ChatOpenAI model for Gemma
llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    temperature=0,
    api_key="not-needed",
    model_name="llama3"
)

In [ ]:
# load the document
#path_file=path_file
loader = UnstructuredFileLoader(path_file)
documents = loader.load()

In [ ]:
path_file="/home/nugh75/git-repository/env1/Nugh75-didattico-repository/doc-qa/pdfs/hoder.pdf"
# load the document
#path_file=path_file
loader = UnstructuredFileLoader(path_file)
documents = loader.load()

In [ ]:
#documents=path_file
type(documents[0])

In [ ]:
# create text chunks

text_splitter = CharacterTextSplitter(separator='/n',
                                      chunk_size=2000,
                                      chunk_overlap=200)

text_chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =500,
    chunk_overlap=200
)
splits =text_splitter.split_documents(documents)
for sp in splits:
    if (len(sp.page_content) <100):
        splits.remove(sp)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [ ]:
cartella="Hoder conoscenza"

In [ ]:
import os
from langchain_community.vectorstores import FAISS

if os.path.exists(cartella):
    #Carica indice FAISS cartella corrente attuale
    faiss_index=FAISS.load_local(
        cartella,
        embeddings,
        allow_dangerous_deserialization=True
    )
else:
    #crea indice FAISS dei chunk nella cartella attuale
    faiss_index = FAISS.from_documents(
        splits,
        embeddings
    )
    faiss_index.save_local(cartella)

In [ ]:
retriever = faiss_index.as_retriever(

    search_type="similarity",
    search_kwargs={"k": 4} 
)

In [ ]:
from langchain import hub
prompt =hub.pull("rlm/rag-prompt")

In [ ]:
intestazione=(documents[0].page_content[0:1000])
#def format_documents(intestazione):
#    return "\n\n".join(doc.page_content for doc in intestazione)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_template("Sei un assistente preciso e attento ; Rispondi a questa domanda in italiano: {question}, quando rispondi dai come prima informazione il titolo del documento di cui si parla e l'autore. Trovi queste informazionie in questo documento alla prima pagina {intestazione}. Dopo per la risposta considra il seguente contesto {context}.  ")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

rag_chain=(
    {
        "context": retriever  | format_documents,
        "question": RunnablePassthrough(),
        "intestazione": intestazione
        }
    | prompt
    | llm
    | StrOutputParser()
    )

In [ ]:
def query(query):
    answer = rag_chain.invoke(query)
    return answer

def queryStream(query):
    for chunk in rag_chain.stream(query):
        print(chunk, end="", flush=True)

In [ ]:
query("riassumi l'articolo")